This notebook provide a technical background of the multi-family apartment building model.

# Thermal Model

The thermal model is a based on a four-zone ETP model, where the following zones variables are defined:

Table 1: Zone variables

| Zone Name            | Temperature | Mass   | Heat   |
| :------------------- | :---------: | :----: | :----: |
| Occupied apartment   | $T_A$       | $C_A$  | $Q_A$  |
| Unoccupied apartment | $T_U$       | $C_U$  | $Q_U$  |
| Building core        | $T_C$       | $C_C$  | $Q_C$  |
| Building mass        | $T_M$       | $C_M$  |   -    |
| Outdoor air          | $T_O$       |   -    |   -    |

The model assumes that every zone is connected to every other zone with the following zone-zone conductance parameters defined

Table 2: Zone-zone conductance

| Conductance | Connected zones                       |
| ----------- | ------------------------------------- |
| $U_{OA}$    | Occupied apartment & outdoor air      |
| $U_{OU}$    | Unoccupied apartment & outdoor air    |
| $U_{OC}$    | Building core & outdoor air           |
| $U_{OM}$    | Building mass & outdoor air           |
| $U_{AU}$    | Occupied & unoccupied apartments      |
| $U_{AC}$    | Occupied apartments & building core   |
| $U_{AM}$    | Occupied apartments & building mass   |
| $U_{UC}$    | Uncccupied apartments & building core |
| $U_{UM}$    | Unoccupied apartments & building mass |
| $U_{CM}$    | Building core & building mass         |

The thermal dynamics are defined by Equation (1)

$$
    \begin{bmatrix} 
        \dot T_A 
    \\ 
        \dot T_U 
    \\
        \dot T_C
    \\
        \dot T_M
    \end{bmatrix}
    =
    \begin{bmatrix}
        -\frac{U_{AU}+U_{AC}+U_{AM}}{C_A} & \frac{U_{AU}}{C_A} & \frac{U_{AC}}{C_A} & \frac{U_{AM}}{C_A}
    \\
        \frac{U_{AU}}{C_U} & -\frac{U_{AU}+U_{UC}+U_{UM}}{C_A} & \frac{U_{UC}}{C_A} & \frac{U_{UM}}{C_A}
    \\
        \frac{U_{AC}}{C_C} & \frac{U_{UC}}{C_C} & -\frac{U_{AC}+U_{UC}+U_{CM}}{C_C} & \frac{U_{CM}}{C_C}
    \\
        \frac{U_{AM}}{C_M} & \frac{U_{UM}}{C_M} & \frac{U_{CM}}{C_M} & -\frac{U_{AM}+U_{UM}+U_{CM}}{C_M}
    \end{bmatrix}
    \begin{bmatrix}
        T_A
    \\
        T_U
    \\
        T_C
    \\
        T_M
    \end{bmatrix}
    +
    \begin{bmatrix}
        \frac{1}{C_A} & 0 & 0 & \frac{U_{OA}}{C_A}
    \\
        0 & \frac{1}{C_U} & 0 & \frac{U_{OU}}{C_U}
    \\
        0 & 0 & \frac{1}{C_C} & \frac{U_{OC}}{C_C}
    \\
        0 & 0 & 0 & \frac{U_{OM}}{C_M}
    \end{bmatrix}
    \begin{bmatrix}
        Q_A
    \\
        Q_U
    \\
        Q_C
    \\
        T_O
    \end{bmatrix}
    \qquad (1)
$$

# Heat Gains

The heat gains for each zones are defined as follows per unit

Table 3: Occupied apartment heat gains (per occupied unit)

| Heat source      | Parameter |
| ---------------- | --------- |
| Central plant    | $Q_{AP}$  | 
| Unit HVAC        | $Q_{AH}$  |
| Solar gain       | $Q_{AS}$  |
| Ventilation gain | $Q_{AV}$  |
| End-use gains    | $Q_{AE}$  |

Table 4: Unoccupied apartment heat gains (per unoccupied unit)

| Heat source      | Parameter |
| ---------------- | --------- |
| Central plant    | $Q_{UP}$  | 
| Unit HVAC        | $Q_{UH}$  |
| Solar gain       | $Q_{US}$  |
| Ventilation gain | $Q_{UV}$  |
| End-use gains    | $Q_{UE}$  |

Table 5: Building core heat gains (per total units)

| Heat source      | Parameter |
| ---------------- | --------- |
| Central plant    | $Q_{CP}$  | 
| Unit HVAC        | $Q_{CH}$  |
| Solar gain       | $Q_{CS}$  |
| Ventilation gain | $Q_{CV}$  |
| End-use gains    | $Q_{CE}$  |

# End-uses

The following end-uses are available in building zones

Table 6: Zone end-uses schedule and gain parameter

| End-use     | Occupied      | Unoccupied | Core       | 
| ----------- | ------------- | ---------- | ---------- |
| Lights      | `lighting`    |     -      | `hallway`  |
| Plugs       | `plugs`       |     -      |     -      |
| Cooking     | `cooking`     |     -      |     -      |
| Dishwashing | `dishwashing` |     -      |     -      |
| Washing     | `washing`     |     -      |     -      |
| Drying      | `drying`      |     -      |     -      |
| Hotwater    | `hotwater`    |     -      |     -      |

# HVAC Controls

The following thermostat setpoints are use to control HVAC

| Zone | $T_{AH}$ | $T_{AC}$ |
| ---------- | -------- | --------- |
| Occupied | $T_{AH}$ | $T_{AC}$ |
| Occupied | $T_{AH}$ | $T_{AC}$ |
| Occupied | $T_{AH}$ | $T_{AC}$ |
